<a href="https://colab.research.google.com/github/uvpatil09/c0mrade/blob/main/all_MiniLM_L6_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pypdf import PdfReader

pdf_path = "/content/sample_data/Leave Policy_Apr-25.pdf"

text_content = ""
with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        text_content += page.extract_text()

# -----------------------------
# 2. Text Chunking
# -----------------------------
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

text_chunks = text_splitter.split_text(text_content)

print(f"Total chunks created: {len(text_chunks)}")

# -----------------------------
# 3. Generate Embeddings
# -----------------------------
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# -----------------------------
# 4. Create FAISS Vector Store
# -----------------------------
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_texts(
    texts=text_chunks,
    embedding=embedder
)

print("FAISS vector store created")

# -----------------------------
# 5. HuggingFace LLM Setup
# -----------------------------
from langchain_huggingface import HuggingFacePipeline
import torch

device = 0 if torch.cuda.is_available() else -1

llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    device=device,
    pipeline_kwargs={"max_new_tokens": 200}
)

print("LLM loaded successfully")

# -----------------------------
# 6. RAG Chain using LCEL
# -----------------------------
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Answer the question ONLY using the following context:\n{context}\n"
        "If the answer is not present, reply with exactly: I don't know"
    ),
    ("human", "{question}")
])

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG chain ready")

# -----------------------------
# 7. Test Questions
# -----------------------------
# In-context question
q1 = "What are the different types of leaves?"
print("\nQ:", q1)
print("A:", rag_chain.invoke(q1))

# Out-of-context question
q2 = "What is the capital of France?"
print("\nQ:", q2)
print("A:", rag_chain.invoke(q2))